# Synthetic data generation

This tutorial shows how to use Sim4Rec to generate synthetic data based on real data. First, we fit non-negative ALS to real data containing user interactions and obtain vector representations of real users. Then, we fit CopulaGAN to non-negative ALS embeddings of real users and generate synthetic user feature vectors with CopulaGAN. We evaluate the quality of the generated synthetic data using the metrics LogisticDetection, SVCDetection, KSTest, and ContinuousKLDivergence.

In [1]:
import os
import sys
os.environ["JAVA_HOME"] = "/home/jovyan/ivanova/env39_sim4rec_check/lib/jvm"
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["OMP_NUM_THREADS"] = "32"
os.environ["NUMBA_NUM_THREADS"] = "4"

In [2]:
import random
import time

import numpy as np
import pandas as pd
from rs_datasets import MovieLens
import warnings

from pyspark import SparkConf
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from sim4rec.modules import evaluate_synthetic, SDVDataGenerator
from sim4rec.utils import pandas_to_spark

warnings.filterwarnings("ignore") 

In [3]:
SPARK_LOCAL_DIR = "./tmp/task_2"

In [4]:
%%bash -s "$SPARK_LOCAL_DIR"
rm -rf $1

In [5]:
spark = SparkSession.builder\
    .appName("simulator")\
    .master("local[4]")\
    .config("spark.sql.shuffle.partitions", "4")\
    .config("spark.default.parallelism", "4")\
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC")\
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC")\
    .config("spark.sql.autoBroadcastJoinThreshold", "-1")\
    .config("spark.driver.memory", "4g")\
    .config("spark.local.dir", SPARK_LOCAL_DIR)\
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/12 21:43:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/12 21:43:18 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
25/03/12 21:43:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/12 21:43:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/12 21:43:18 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/03/12 21:43:18 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/03/12 21:43:18 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/03/12 21:43:18 WARN Utils: Servi

In [6]:
SEED = 1234
np.random.seed(SEED)

## Download Data

In [7]:
data = MovieLens("20m")
ratings_df = data.ratings
ratings_df.head()

,user_id,item_id,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [8]:
rating_threshold = ratings_df["rating"].median()
rating_threshold

3.5

In [9]:
ratings_df["relevance"] = (ratings_df["rating"] >= rating_threshold).astype(int)

## Data Preprocessing

We encode the user and item IDs with consecutive values so that the new IDs do not contain missing values.

In [10]:
ratings_df["user_idx"] = ratings_df["user_id"].astype("category").cat.codes
ratings_df["item_idx"] = ratings_df["item_id"].astype("category").cat.codes

In [11]:
train = pandas_to_spark(ratings_df)

## Non-negative ALS for User Embeddings

Fit non-negative ALS to historical data to produce the real user vectors that represent their profiles.

In [12]:
# initialization of non-negative ALS
als = ALS(
    rank=64,
    maxIter=5,
    userCol="user_idx",
    itemCol="item_idx",
    ratingCol="relevance",
    seed=SEED,
    nonnegative=True,
)
# fit ALS
als_model = als.fit(train)

Obtain vector representations of real users.

In [13]:
user_features = als_model.userFactors.orderBy("id")
user_features = (user_features.withColumn("user_feature", col("features"))).select(
    ["id"] + [col("user_feature")[i] for i in range(64)]
)
user_features.limit(5).toPandas()

,id,user_feature[0],user_feature[1],user_feature[2],user_feature[3],user_feature[4],user_feature[5],user_feature[6],user_feature[7],user_feature[8],...,user_feature[54],user_feature[55],user_feature[56],user_feature[57],user_feature[58],user_feature[59],user_feature[60],user_feature[61],user_feature[62],user_feature[63]
0,0,0.098465,0.052241,0.125119,0.202401,0.178767,0.148965,0.118857,0.080273,0.152939,...,0.077556,0.187669,0.081136,0.129824,0.078120,0.145179,0.096780,0.099917,0.154622,0.113657
1,1,0.083297,0.047674,0.103928,0.181666,0.161923,0.128280,0.096112,0.068772,0.143337,...,0.059835,0.174903,0.070022,0.122740,0.070366,0.123795,0.083381,0.087477,0.137442,0.099442
2,2,0.089750,0.044578,0.118326,0.188298,0.157684,0.135538,0.109513,0.071072,0.142481,...,0.074062,0.166076,0.075291,0.117528,0.068250,0.135233,0.085594,0.092989,0.142653,0.100441
3,3,0.065662,0.048028,0.068513,0.146848,0.143943,0.101966,0.062271,0.059757,0.128780,...,0.031322,0.164175,0.050798,0.111714,0.065364,0.094431,0.072750,0.065016,0.117407,0.087755
4,4,0.076065,0.054559,0.085825,0.180649,0.170346,0.122832,0.076311,0.067553,0.156395,...,0.041212,0.192011,0.060475,0.133150,0.074588,0.115542,0.083266,0.077581,0.139359,0.101348


## CopulaGAN User Embeddings Generator

Fit the SDVDataGenerator with the CopulaGAN synthetic data model to non-negative ALS embeddings of real users and generate synthetic user feature vectors.

In [14]:
# initialization of data generator
sdv_data_generator = SDVDataGenerator(
    label="synth",
    id_column_name="user_id",
    model_name="copulagan",
    parallelization_level=4,
    device_name="cpu",
    seed=SEED,
)
# fit data generator
sdv_data_generator.fit(user_features.drop("id").sample(0.1))
# generate user embeddings
synthetic_users = sdv_data_generator.generate(user_features.sample(0.1).count())
synthetic_users.limit(5).toPandas()

,user_id,user_feature[0],user_feature[1],user_feature[2],user_feature[3],user_feature[4],user_feature[5],user_feature[6],user_feature[7],user_feature[8],...,user_feature[54],user_feature[55],user_feature[56],user_feature[57],user_feature[58],user_feature[59],user_feature[60],user_feature[61],user_feature[62],user_feature[63]
0,synth_0,0.048689,3.835446e-08,0.043500,0.008521,0.000422,0.014227,0.029316,0.018899,0.107306,...,0.042797,0.120833,0.030708,0.000873,0.058941,0.100261,0.046083,0.007793,0.053728,0.019363
1,synth_1,0.069267,1.930680e-02,0.124266,0.113976,0.034785,0.079196,0.107183,0.028989,0.087186,...,0.093359,0.095578,0.066002,0.034499,0.021036,0.111568,0.052000,0.053175,0.108334,0.054314
2,synth_2,0.083154,3.803321e-02,0.107095,0.161039,0.116115,0.125890,0.075523,0.053697,0.137838,...,0.031829,0.096073,0.054047,0.090792,0.059796,0.121177,0.073694,0.082214,0.101911,0.063838
3,synth_3,0.025873,2.291479e-02,0.007333,0.122892,0.109143,0.056939,0.018374,0.038897,0.124322,...,0.032198,0.159394,0.058620,0.054539,0.060978,0.092050,0.062194,0.002803,0.106136,0.074685
4,synth_4,0.069708,2.341466e-03,0.086818,0.051152,0.086035,0.040961,0.097504,0.047911,0.077176,...,0.045301,0.114128,0.051297,0.003335,0.037135,0.112214,0.001924,0.063257,0.063910,0.026804


## Generator Evaluation

We evaluate the quality of synthetic user profiles using the following metrics:

$LogisticDetection = 1 - ROC\text{-}AUC_{LR}$ 

Evaluates how difficult it is to separate the real and synthetic data using the Logistic Regression Model. 

The less, the better.

$SVCDetection = 1 - ROC\text{-}AUC_{SVC}$ 

Evaluates how difficult it is to separate the real and synthetic data using the Support Vector Classifier. 

The less, the better.

$KSTest = 1 - sup_x|CDF_{real} - CDF_{synth}|$ 

Analyzes the similarity of distributions of continuous-valued attributes in real and synthetic data using the Kolmogorov-Smirnov two-sample test. 

The more, the better.

$ContinuousKLD = \frac{1}{1 + KLD}$ 

Evaluates the distance of probability distributions of continuous-valued attributes in real and synthetic data. 

The more, the better.

In [15]:
# sample users
real_users = user_features.sample(0.1)
# generate synthetic users
synthetic_users = sdv_data_generator.generate(real_users.count())
# evaluate the quality of synthetic data
gen_score = evaluate_synthetic(
    synthetic_users.drop("user_id"), 
    real_users.drop("id")
)
gen_score

{'LogisticDetection': 0.004599835168113686,
 'SVCDetection': 6.247331633757458e-05,
 'KSTest': 0.8917719533443049,
 'ContinuousKLDivergence': 0.47807569851046233}

## Sample Users

In [16]:
synthetic_users.limit(5).toPandas()

,user_id,user_feature[0],user_feature[1],user_feature[2],user_feature[3],user_feature[4],user_feature[5],user_feature[6],user_feature[7],user_feature[8],...,user_feature[54],user_feature[55],user_feature[56],user_feature[57],user_feature[58],user_feature[59],user_feature[60],user_feature[61],user_feature[62],user_feature[63]
0,synth_0,0.077470,0.007660,0.055656,0.127461,0.057922,0.066215,0.093525,0.043931,0.119463,...,0.061100,0.164211,0.046135,0.034788,0.047786,0.114947,0.043772,0.039991,0.104441,0.047943
1,synth_1,0.102026,0.036502,0.135215,0.185076,0.117268,0.134396,0.132147,0.065629,0.126947,...,0.100680,0.095686,0.091965,0.072183,0.070429,0.138953,0.094389,0.089172,0.109634,0.083437
2,synth_2,0.080357,0.006699,0.109054,0.169239,0.093512,0.107995,0.132497,0.064672,0.123864,...,0.078298,0.102557,0.078328,0.076340,0.036955,0.142110,0.061190,0.094490,0.139088,0.072340
3,synth_3,0.068160,0.005491,0.099192,0.133237,0.059503,0.081220,0.102545,0.036083,0.081888,...,0.085265,0.101910,0.066624,0.029122,0.052093,0.148417,0.050333,0.063438,0.101976,0.046728
4,synth_4,0.109875,0.021703,0.131302,0.196359,0.107740,0.157976,0.140915,0.070884,0.161021,...,0.085384,0.135868,0.086393,0.060904,0.069378,0.134564,0.092904,0.105829,0.166766,0.053991
